# Llamar codigo R desde Python

## Cargar la funcion desde el archivo training.R

In [2]:
import rpy2.robjects as robjects
training = robjects.r.source("./training.R", encoding="utf-8")[0]
training

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7fc962189ec0> [RTYPES.CLOSXP]
R classes: ('function',)

## Mostrar el codigo de la funcion en R

In [3]:
print(training.r_repr())

function (name) 
{
    model <- mlflow_load_model(model_uri = name)
    return(model)
}


## Ejecutar la funcion con un input

In [4]:
get_model_R = robjects.r("get_model")
get_model_R

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7fc96c866b40> [RTYPES.CLOSXP]
R classes: ('function',)

In [5]:
model_name = "models:/model1_R/1"
model_R = get_model_R(model_name)
model_R

handle,[RTYPES.EXTPTRSXP]
raw,[RTYPES.RAWSXP]
niter,[RTYPES.INTSXP]


In [10]:
aux = model_R[0]
aux

<rpy2.rinterface.SexpExtPtr object at 0x7f7c12a8d5c0> [RTYPES.EXTPTRSXP]

In [6]:
%load_ext rpy2.ipython

In [7]:
%%R

#R -i df
R.version.string

[1] "R version 4.2.2 (2022-10-31)"


In [8]:
%%R

library(mlflow)
Sys.setenv(MLFLOW_BIN=system("which mlflow", intern=TRUE))
Sys.setenv(MLFLOW_PYTHON_BIN=system("which python", intern=TRUE))

get_model <- function(name) {
    model <- mlflow_load_model(model_uri=name)
    return(model)
}
model_name <- "models:/model1_R/1"
get_model(model_name)

##### xgb.Booster
raw: 88 Kb 
xgb.attributes:
  niter
niter: 99


In [10]:
model_name = "models:/model1_R/1"
get_model_R = robjects.r['get_model']
get_model_R(model_name)

handle,[RTYPES.EXTPTRSXP]
raw,[RTYPES.RAWSXP]
niter,[RTYPES.INTSXP]


# From scratch

In [1]:
import rpy2.robjects as robjects

In [2]:
%%time

import rpy2.robjects as robjects


robjects.r("""
            library(xgboost)
            library(mlflow)
            Sys.setenv(MLFLOW_BIN=system("which mlflow", intern=TRUE))
            Sys.setenv(MLFLOW_PYTHON_BIN=system("which python", intern=TRUE))

            Tablon_test=read.table(file='training/x_test.csv',sep=',',header=T,stringsAsFactors=FALSE)
            dtest_M1 <- xgb.DMatrix(data=as.matrix(Tablon_test),missing=NA)

            name <- "models:/model1_R/1"
            model <- mlflow_load_model(model_uri=name)

            predicted=predict(model,dtest_M1,missing=NA)
        """)

robjects.r['predicted']


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    CPU times: user 4.63 s, sys: 161 ms, total: 4.79 s
Wall time: 4.47 s


0.000018,0.001825,-0.000003,...,-0.000065,-0.000003,-0.000035


In [39]:
%%R

library(xgboost)
library(mlflow)
Sys.setenv(MLFLOW_BIN=system("which mlflow", intern=TRUE))
Sys.setenv(MLFLOW_PYTHON_BIN=system("which python", intern=TRUE))

Tablon_test=read.table(file='x_test.csv',sep=',',header=T,stringsAsFactors=FALSE)
dtest_M1 <- xgb.DMatrix(data=as.matrix(Tablon_test),missing=NA)

name <- "models:/model1_R/1"
model <- mlflow_load_model(model_uri=name)

predicted=predict(model,dtest_M1,missing=NA)
predicted[1:10]

 [1]  1.798484e-05  1.824704e-03 -3.037206e-06 -5.303383e-05  3.638420e-05
 [6] -5.325146e-07  4.433680e-05 -3.803310e-05 -2.574593e-04  1.798484e-05
